In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ejercicio 4. Estrellas


## Inciso A

In [2]:
data = pd.read_csv(r'.\estrellas.csv')
data

In [3]:
# ver rango de columna Edad
print(data['Edad'].min())
print(data['Edad'].max())
print(data['Edad'].mean())
print(data['Edad'].median()) 

In [4]:
data['Edad_bin'], bins = pd.qcut(data['Edad'], q=2, labels=['Baja', 'Alta'], retbins=True)

In [5]:
data

In [6]:
print('Rangos de los intervalos: ', bins)
conteo_intervalos = data['Edad_bin'].value_counts()
print('Cantidad de ejemplos en cada intervalo:\n', conteo_intervalos)

# 5. Ejercicio Integrador: Ofertas de Trabajo 

In [7]:
df = pd.read_excel(r'trabajos.xlsx')
df

## Inciso A

In [8]:
# Mostrar solo las columnas Job Title y Job Description
df[['Job Title', 'Job Description']] 

## Inciso B

In [9]:
df.columns

In [10]:
# Eliminar columnas index, Competitors, Company Name
df = df.drop(['index', 'Competitors', 'Company Name'], axis=1)

In [11]:
df

In [12]:
df.describe()

## Inciso C
### a) Crear columna de sueldo minimo

In [13]:
# Crear columna de sueldo minimo y máximo
df['Salary Estimate'].unique()


In [14]:
# Eliminar el texto de la columna
df['Salary Estimate'] = df['Salary Estimate'].str.replace(' (Glassdoor est.)', '')
# Crear columna de sueldo minimo
df['Salary Min'] = df['Salary Estimate'].str.split('-').str[0].str.replace('$', '').str.replace('K', '').astype('float')

In [15]:
df['Salary Min'].describe()
#print(df['Salary Min'].value_counts())

### b) Crear columna de sueldo maximo

In [16]:
# Eliminar el texto de la columna
df['Salary Estimate'] = df['Salary Estimate'].str.replace(' (Glassdoor est.)', '').str.replace('(Employer est.)', '')
# Crear columna de sueldo maximo
df['Salary Max'] = df['Salary Estimate'].str.split('-').str[1].str.replace('$', '').str.replace('K', '').astype('float')

In [17]:
df['Salary Max'].describe()

### c) Crear columna de sueldo promedio

In [18]:
# Crear columna de salario promedio
df['Salary Avg'] = (df['Salary Min'] + df['Salary Max']) / 2
df

In [19]:
df = df.drop(columns=['Salary Estimate'])

### d) Crear columna de Ubicación 
Del trabajo, codificada como el código del estado de dos letras (por ej, NY o MA), sin la ciudad. 
Quitar el resto de los atributos de ubicación (Location y Headquarters)

In [20]:
df['Location'].unique()

In [21]:
df['Ubicacion'] = df['Location'].apply(lambda x: x.split(',')[1].strip() if ',' in x else x)


In [22]:
df['Ubicacion'].unique()

In [23]:
# Cambiar los valores de Ubicacion de New Jersey a NJ, California a CA, Texas a TX, etc. Quedó medio manual pero no le encontré otra forma
df['Ubicacion'] = df['Ubicacion'].replace({'New Jersey': 'NJ', 'California': 'CA', 'Texas': 'TX', 'United States': 'US', 'Utah': 'UT', "Anne Arundel": "MD"})  
df['Ubicacion'].unique()

In [24]:
df = df.drop(['Location'], axis=1)
df = df.drop(['Headquarters'], axis=1)

### e) Antigüedad de la empresa
Codificada como número, en base a la Fecha de fundación. 

Quitar la fecha de fundación.


In [25]:
# Columna de  Antiguedad de la empresa
df['Antiguedad'] = (2025 - df['Founded']).astype('int')

In [26]:
# Eliminar la fecha de fundacion
df = df.drop(['Founded'], axis=1)

In [27]:
df

### f) Senior: (binaria)
Si el trabajo es para una persona con mucha experiencia (atributo binario)

Nota: Para identificar si un trabajo es “Senior”, consultar si los atributos
“Job Title” o “Job Description” contienen las palabras: “Senior”, “Sr”, “Sr.”,
¿Qué otras palabras se te ocurren para mejorar este atributo?


In [28]:
# Crear columna binaria (booleana?) de Seniority, segun si aparece la palabra Senior o Sr. en la columna Job Title y Job Description
palabras_clave = 'Senior|Sr.|Sr|Lead|Expert|Principal|Director|Chief'
df['Seniority'] = df['Job Title'].str.contains(palabras_clave)
df['Seniority'] = df['Seniority'] | df['Job Description'].str.contains(palabras_clave)
df['Seniority'] = df['Seniority'].astype('int') # No se si poner tipo booleano o entero

# Hay varias palabras (como Expert / Lead / Principal, etc) que tambien podrian indicar Seniority

In [29]:
df

### g) Junior: (binaria)
si el trabajo es para una persona con poca experiencia (atributo binario).

Nota: Para identificar si un trabajo es “Junior”, consultar si los atributos
“Job Title” o “Job Description” contienen las palabras: “Junior” o “Jr”. ¿Qué
otras palabras se te ocurren para mejorar este atributo

In [30]:
# Crear columna Junior segun si aparece la palabra Junior o Jr. en la columna Job Title y Job Description
palabras_clave = 'Junior|Jr.|Jr|Early|Entry|Beginner'
df['Junior'] = df['Job Title'].str.contains(palabras_clave)
df['Junior'] = df['Junior'] | df['Job Description'].str.contains(palabras_clave)
df['Junior'] = df['Junior'].astype('int') # No se si poner tipo booleano o entero acá tmp

In [31]:
df

Revisando, veo que varios que estan marcados como Junior, también están como Seniority. Algo está mal evidentemente (quizas haya q cambiar las palabras clave)

### h) “Public” y “Private”
Atributos binarios en base al atributo “Type of ownership”. 
Eliminar ese atributo

In [32]:
df['Type of ownership'].unique()

In [33]:
# Rellenar NaN con una cadena vacía para evitar errores
df['Type of ownership'] = df['Type of ownership'].fillna('')
# Crear columna de si la empresa es privada o no segun si aparece la palabra Private en la columna Type of ownership
df['Private'] = df['Type of ownership'].str.contains('Private|Self-employed').fillna(False).astype('int')
df['Public'] = df['Type of ownership'].str.contains('Public|Government').fillna(False).astype('int')
# Los demás no sé si cómo clasificarlos

Faltaron clasificar algunos

In [34]:
df

### i) “Estimated size” y “Estimated revenue” 
en base a “Size” y “Revenue”, pero los nuevos atributos deben ser numéricos, y deben corresponder al
máximo de los valores que aparecen de forma textual en el atributo. 

Por ejemplo, si el valor textual es 100 to 500 million (USD), el valor numérico debería ser 500.000.000. 
En el caso de empresas que no tienen este valor, reemplazar por el promedio.


In [35]:
# Nueva Columna Estimated Size (maximo que aparece en la columna Size que es de tipo texto)
df['Size'].unique()
# Supongamos que tu DataFrame se llama df y la columna es 'Size'
def extract_max_size(size):
    if isinstance(size, str) and 'to' in size:
        # Dividir la cadena y seleccionar el tercer elemento
        max_size = size.split()[2]
        # Eliminar cualquier carácter no numérico
        max_size = ''.join(filter(str.isdigit, max_size))
        return int(max_size)
    else:
        # Si no hay 'to', verificar si contiene la palabra 'employees'
        if isinstance(size, str) and 'employees' in size:
            return 10000
        else:
            return 0

# Aplicar la función a la columna 'Size' y crear la nueva columna 'Estimated Size'
df['Estimated Size'] = df['Size'].apply(extract_max_size)

In [36]:
# Calcular promedio de Estimated Size sin tener en cuenta los 0
prom_size = round(df['Estimated Size'][df['Estimated Size'] != 0].mean(),0)
# Rellenar los 0 con el promedio
df['Estimated Size'] = df['Estimated Size'].replace(0, prom_size)

In [37]:
df['Revenue'].unique()

In [38]:
def extract_max_rev(size):
    if isinstance(size, str) and 'to' in size:
        max_rev = size.split('to')[1]
        # Eliminar cualquier caracter no numerico
        max_rev = ''.join(filter(str.isdigit, max_rev))
        # Si size contiene la palabra million, devolver el valor por e6, si contiene billion por e9
        if 'million' in size:
            return int(max_rev) * 10**6
        elif 'billion' in size:
            return int(max_rev) * 10**9
        else:
            return 'Problemas CHEQUEAR'
        
    elif isinstance(size, str) and '$' in size:
        max_rev = size.split('$')[1]
        # Eliminar cualquier caracter no numerico
        max_rev = ''.join(filter(str.isdigit, max_rev))
        # Si size contiene la palabra million, devolver el valor por e6, si contiene billion por e9
        if 'million' in size:
            return int(max_rev) * 10**6
        elif 'billion' in size:
            return int(max_rev) * 10**9
        else:
            return 'Problemas CHEQUEAR 2'
    else:      
        return 0


# Crear la columna Estimated Revenue aplicando la funcion extract_max_rev
df['Estimated Revenue'] = df['Revenue'].apply(extract_max_rev)




In [39]:
# Calcular promedio de Estimated Revenue sin tener en cuenta los valores 0
rev_prom = round(df['Estimated Revenue'][df['Estimated Revenue'] != 0].mean(), 0)
# Reemplazar los valores 0 de Estimated Revenue con el promedio
df['Estimated Revenue'] = df['Estimated Revenue'].replace(0, rev_prom)

In [40]:
# Chequeo viendo la cantidad de desconocidos y 0:
print(df['Revenue'].value_counts())
print(df['Estimated Revenue'].value_counts())

In [41]:
df

### j) 10 Atributos Binarios con Palabras Clave
Los títulos de los trabajos contienen mucha variación y en algunos casos
palabras que no contribuyen a la descripción o son muy específicas. Cree
10 atributos binarios con las palabras o frases clave más comunes, como
“Data”, “Engineer” o “Scientist”, y elimine el atributo “Job Title”.

In [45]:
# Crear una lista de palabras clave comunes
keywords = ['Data', 'Engineer', 'Scientist', 'Analyst', 'Manager', 
            'Developer', 'Consultant', 'Architect', 'Specialist', 'Administrator']

# Crear nuevas columnas binarias para cada palabra clave
for keyword in keywords:
    df[keyword] = df['Job Title'].str.contains(keyword, case=False, na=False).astype(int)

# Eliminar la columna original 'Job Title'
df = df.drop('Job Title', axis=1)

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())

### k) Las decripciones de los trabajos están en lenguaje natural y es dificl utilizarlas en un proceso de mineria.
Identicar 10 habilidades muy ocurrentes que se mencionan en las descripciones y crear un atributor binario por cada una. 

In [47]:
# Lista de habilidades (?)
skills = ['Python', 'SQL', 'Machine Learning', 'Deep Learning', 'Excel', 
          'Communication', 'Leadership', 'Cloud', 'Big Data', 'Statistics']

# Crear nuevas columnas binarias para cada habilidad
for skill in skills:
    df[skill] = df['Job Description'].str.contains(skill, case=False, na=False).astype(int)

# Mostrar las primeras filas del DataFrame para verificar
print(df.head())

### l) Convierta los valores del atributo "Industry" que tengan menos de 20 ocurrencias a "Other"

In [49]:
# Contar las ocurrencias de cada valor en la columna 'Industry'
industry_counts = df['Industry'].value_counts()

# Reemplazar los valores con menos de 20 ocurrencias por "Other"
df['Industry'] = df['Industry'].apply(lambda x: 'Other' if industry_counts[x] < 20 or x == -1 else x)

# Mostrar las primeras filas para verificar
print(df['Industry'].value_counts())

## D) Elimine los atributos "'Job Title' y 'Job Description'
'Job Title' ya lo eliminé antes

In [50]:
df = df.drop('Job Description', axis=1)


##  E) Elimine ejemplos con valores faltantes

Ya los estuve cambiando por otros valores anteriormente, pero igual dejo el codigo escrito

In [51]:
missing_values_count = df.isnull().sum().sum()
print(f"Cantidad de ejemplos con valores faltantes: {missing_values_count}")

In [52]:
# Eliminar ejemplos con valores faltantes
df = df.dropna()

## F) Elimine ejemplos duplicados

In [53]:
cantidad_repetidos = df.duplicated().sum()
print(f"Cantidad de ejemplos repetidos: {cantidad_repetidos}")

In [54]:

# Eliminar ejemplos repetidos
df = df.drop_duplicates()
repetidos = df.duplicated().sum()
print(f"Cantidad de ejemplos repetidos luego de eliminar: {repetidos}")

## G ) Experimente y elija 3 graficos complementarios distintos
que les parezca que puedan describir al conjunto de datos de foirma visual y resumida

### Grafico de barras

In [55]:
# Ejemplo: Distribución de la columna 'Industry'
df['Industry'].value_counts().plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Distribución de la Industria')
plt.xlabel('Industria')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.show()

## Grafico de dispersión

In [64]:
plt.figure(figsize=(10, 6))
plt.scatter(df['Estimated Size'], df['Estimated Revenue'], alpha=0.7, color='green')

plt.xlabel('Tamaño estimado')
plt.ylabel('Salario estimado')
plt.grid(True)
plt.show()

### Caja de Tukey

In [68]:
# Calcular los límites de Tukey
q1 = df['Antiguedad'].quantile(0.25)  # Primer cuartil
q3 = df['Antiguedad'].quantile(0.75)  # Tercer cuartil
iqr = q3 - q1  # Rango intercuartílico

# Límites para los bigotes
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filtrar valores dentro de los límites
filtered_data = df[(df['Antiguedad'] >= lower_bound) & (df['Antiguedad'] <= upper_bound)]

# Crear el boxplot
plt.figure(figsize=(10, 6))
plt.boxplot(filtered_data['Antiguedad'], vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue'))
plt.title('Boxplot de Tukey para Antigüedad')
plt.xlabel('Antigüedad')
plt.show()

Identifiqué un problema en la columna antiguedad, con algunos valores extremos mayores. Faltaria corregir eso.